In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
from redshift_credentials import database, username, password, host, port, schema
from sqlalchemy.orm import sessionmaker
import itertools
import pygsheets as pygs
import datetime
from datetime import timedelta, date
from pandas.tseries.offsets import MonthEnd
import calendar
from dateutil.relativedelta import relativedelta

In [2]:
def sql_con(in_username, in_password, in_host, in_port, in_database, in_schema):
    '''Create connection function'''
    connection = "redshift+psycopg2://%s:%s@%s:%s/%s" % (in_username, in_password, in_host, str(in_port), in_database)
    engine = sa.create_engine(connection)
    session = sessionmaker()
    session.configure(bind=engine)
    s = session()
    setpath = "SET search_path TO %s" % in_schema
    s.execute(setpath)
    return engine

In [3]:
g_drive_cred = r'C:\Users\darius kay\Dev\gitlab\test\python\credentials.json'
# Main dashboard
workbook1 = "https://docs.google.com/spreadsheets/d/1Y6D0-TRXMT3gwN3HITouUmMJrhUPqItTykXcCFfU23I/edit#gid=1763802414"
# Reason dashboard
workbook2 = "https://docs.google.com/spreadsheets/d/1mZUH-iNi3N2uAYlCUP-sH47hlGZxSy-yIJ-NHz5oaVk/edit#gid=408816025"
# Style dashboard.
workbook3 = 'https://docs.google.com/spreadsheets/d/1oV3y4aTKGDEmllsvn4Aq3mMPqJeA3F2yko8JX8tVIIg/edit#gid=555002914'
# Replacement reason dashboard.
workbook4 = 'https://docs.google.com/spreadsheets/d/1oNdsNlaXkCSW1I4zDPy8Xm6K0eRqlJMp3GhPYp5H6ik/edit#gid=444915816'
# Replacement style dashboard.
workbook5 = 'https://docs.google.com/spreadsheets/d/1A5KyqmYzNpZLI2nHajNmMg4zTg7_2CzEby2GI6UbB2k/edit#gid=555002914'
# Recovery dashboard.
workbook6 = 'https://docs.google.com/spreadsheets/d/1sTYoZ85uQXQdSTDyEHIs7naSTgGfKIg3lD4oO3oZj8Q/edit#gid=1059987213'

# Function to clear data range.
def clear_data_range(workbook_name, tab_name):
#     print(g_drive_cred)
    # authorization
    gc = pygs.authorize(client_secret=g_drive_cred)
    # open the google spreadsheet
    ws = gc.open_by_url(workbook_name)
    # select the first sheet
    wks = ws.worksheet_by_title(tab_name)
#     print(ws.id)
    wks.clear()
    print(f"{workbook_name} - {tab_name}: Cleared")
    return

def pd_to_gsheets(workbook_name, sheet_name, df, row_num, col_num):
    # authorization
    gc = pygs.authorize(client_secret=g_drive_cred)

    # open the google spreadsheet
    ws = gc.open_by_url(workbook_name)

    # select the first sheet
    wks = ws.worksheet_by_title(sheet_name)
    
    # Determine row count
    row_count = wks.rows
    print(f"Rows available in WS: {row_count}")
    # Determine length of DF.
    new_row_count = len(df)
    print(f"Rows in DF: {new_row_count}")
    # Determine if we need more rows.
    if new_row_count > row_count:
        diff = new_row_count - row_count
        # Add additional rows if needed.
        wks.add_rows(diff)
        print(f"{diff} rows added to WS")
    
    # update the designated sheet with df, starting at cell B2.
    wks.set_dataframe(df, (row_num, col_num))

    return

In [4]:
def date_checker(view, date, schedule):
    if view == 'forecast':
        if schedule == 'week':
            try:
                return date + timedelta(days=21)
            except TypeError:
                return
    else:
        return date

In [5]:
# Define connection.
conn = sql_con(username, password, host, port, database, schema)

In [6]:
### Calculate current EOW
current_eow = datetime.date.today() - timedelta(days=datetime.date.today().weekday()) + timedelta(days=6)
### Determine limits per query.
starting_55_eow = current_eow - timedelta(weeks=56)
# Get last month
last_day = calendar.monthrange(int(datetime.date.today().strftime("%Y")), int(datetime.date.today().strftime("%m")))[1]
current_eom = datetime.date(year=datetime.date.today().year, day=last_day, month=datetime.date.today().month)
last_month = current_eom - relativedelta(months=+13)
last_month

datetime.date(2019, 12, 31)

In [7]:
# Primary weekly forecast query.
weekly_forecast_query = "SELECT fulfillment_week_ended week_ended, %s, %s qty_returned, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold, qty_returned * 1.0 / NULLIF(qty_sold, 0) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', fulfillment_week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 2 AND fulfillment_week_ended <= CAST(DATE_TRUNC('week',GETDATE())+INTERVAL '1 weeks' -1 AS DATE)"

weekly_forecast_category_query = "SELECT a.week_ended, a.view, a.product, a.send_back__type, a.plant, c.master_category, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_week_ended week_ended, %s, SUM(CASE WHEN return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_week_ended week_ended, %s, CASE WHEN return_receive_date IS NOT NULL AND master_category IS NULL THEN 'Other' ELSE master_category END master_category, SUM(CASE WHEN return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.week_ended = c.week_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.plant = c.plant WHERE DATE_PART('year', a.week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 2 AND a.week_ended <= CAST(DATE_TRUNC('week',GETDATE())+INTERVAL '1 weeks' -1 AS DATE)"

weekly_forecast_reason_query = "SELECT a.week_ended, a.view, a.product, a.send_back__type, a.size, c.master_reason, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_week_ended week_ended, %s, SUM(CASE WHEN return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_week_ended week_ended, %s, CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NULL THEN 'Other' ELSE master_reason END master_reason, SUM(CASE WHEN return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.week_ended = c.week_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.size = c.size WHERE DATE_PART('year', a.week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 2 AND a.week_ended <= CAST(DATE_TRUNC('week',GETDATE())+INTERVAL '1 weeks' -1 AS DATE)"

# Actual weekly primary query.
weekly_actual_query = "SELECT fulfillment_week_ended week_ended, %s, %s qty_returned, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold, qty_returned * 1.0 / NULLIF(qty_sold, 0) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', fulfillment_week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 2"

weekly_actual_category_query = "SELECT a.week_ended, a.view, a.product, a.send_back__type, a.plant, c.master_category, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_week_ended week_ended, %s, SUM(CASE WHEN return_receive_date IS NOT NULL THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_week_ended week_ended, %s, CASE WHEN return_receive_date IS NOT NULL AND master_category IS NULL THEN 'Other' ELSE master_category END master_category, SUM(CASE WHEN return_receive_date IS NOT NULL THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.week_ended = c.week_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.plant = c.plant WHERE DATE_PART('year', a.week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 2"

weekly_actual_reason_query = "SELECT a.week_ended, a.view, a.product, a.send_back__type, a.size, c.master_reason, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_week_ended week_ended, %s, SUM(CASE WHEN return_receive_date IS NOT NULL THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_week_ended week_ended, %s, CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NULL THEN 'Other' ELSE master_reason END master_reason, SUM(CASE WHEN return_receive_date IS NOT NULL THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.week_ended = c.week_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.size = c.size WHERE DATE_PART('year', a.week_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 2"

In [8]:
# Monthly actual primary query.
monthly_actual_query = "SELECT fulfillment_month_ended month_ended, %s, %s qty_returned, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold, qty_returned * 1.0 / NULLIF(qty_sold, 0) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', fulfillment_month_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 2"

monthly_actual_category_query = "SELECT a.month_ended, a.view, a.product, a.send_back__type, a.plant, c.master_category, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, %s, SUM(CASE WHEN return_receive_date IS NOT NULL THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_month_ended month_ended, %s, CASE WHEN return_receive_date IS NOT NULL AND master_category IS NULL THEN 'Other' ELSE master_category END master_category, SUM(CASE WHEN return_receive_date IS NOT NULL THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.month_ended = c.month_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.plant = c.plant WHERE DATE_PART('year', a.month_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 2"

monthly_actual_reason_query = "SELECT a.month_ended, a.view, a.product, a.send_back__type, a.size, c.master_reason, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, %s, SUM(CASE WHEN return_receive_date IS NOT NULL THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT fulfillment_month_ended month_ended, %s, CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NULL THEN 'Other' ELSE master_reason END master_reason, SUM(CASE WHEN return_receive_date IS NOT NULL THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.month_ended = c.month_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.size = c.size WHERE DATE_PART('year', a.month_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 2"

# Primary monthly forecast query.
monthly_forecast_query = "SELECT LAST_DAY(fulfillment_week_ended + 21) month_ended, %s, %s qty_returned, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold, qty_returned * 1.0 / NULLIF(qty_sold, 0) rate FROM product_pull.ret_exch_rep_data_final GROUP BY 1%s HAVING DATE_PART('year', LAST_DAY(fulfillment_week_ended + 21)) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 2"

monthly_forecast_category_query = "SELECT a.month_ended, a.view, a.product, a.send_back__type, a.plant, c.master_category, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT LAST_DAY(fulfillment_week_ended + 21) month_ended, %s, SUM(CASE WHEN return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT LAST_DAY(fulfillment_week_ended + 21) month_ended, %s, CASE WHEN return_receive_date IS NOT NULL AND master_category IS NULL THEN 'Other' ELSE master_category END master_category, SUM(CASE WHEN return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s AND plant IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.month_ended = c.month_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.plant = c.plant WHERE DATE_PART('year', a.month_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 2"

monthly_forecast_reason_query = "SELECT a.month_ended, a.view, a.product, a.send_back__type, a.size, c.master_reason, c.qty_returned, a.qty_sold, c.qty_returned * 1.0/NULLIF(a.qty_sold, 0) rate FROM (SELECT LAST_DAY(fulfillment_week_ended + 21) month_ended, %s, SUM(CASE WHEN return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT DISTINCT b.* FROM (SELECT LAST_DAY(fulfillment_week_ended + 21) month_ended, %s, CASE WHEN return_receive_date IS NOT NULL AND master_reason IS NULL THEN 'Other' ELSE master_reason END master_reason, SUM(CASE WHEN return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END) qty_returned FROM product_pull.ret_exch_rep_data_final b WHERE send_back__type IN %s GROUP BY 1, 2, 3, 4, 5, 6) b) c ON a.month_ended = c.month_ended AND a.view = c.view AND a.product = c.product AND a.send_back__type = c.send_back__type AND a.size = c.size WHERE DATE_PART('year', a.month_ended) >= DATE_PART_YEAR(TRUNC(GETDATE())) - 2"

In [9]:
# Weekly Forecasted qty returned/exchanged query.
weekly_forecast_qty = "SUM(CASE WHEN send_back__type IN %s AND plant IN %s AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END)"

weekly_forecast_qty_style = "SUM(CASE WHEN send_back__type IN %s AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END)"

# Monthly Forecasted qty returned/exchanged query.
monthly_forecast_qty = "SUM(CASE WHEN send_back__type IN %s AND plant IN %s AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END)"

monthly_forecast_qty_style = "SUM(CASE WHEN send_back__type IN %s AND return_created_date <= fulfillment_week_ended+21 THEN 1 ELSE 0 END)"

# Actual qty returned/exchanged query.
actual_qty = "SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type in %s AND plant IN %s THEN 1 ELSE 0 END)"

actual_qty_style = "SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type in %s THEN 1 ELSE 0 END)"

In [10]:
# Case for product categories.
product_categories = "CASE WHEN shape = 'Rectangle' AND size NOT ILIKE ('%9%x%12%') AND texture = 'Chenille' AND purpose != 'Outdoor' THEN 'rectangular_chenille' WHEN texture = 'Shag' THEN 'shag' WHEN shape = 'Round' AND texture != 'Rubber' THEN 'round' WHEN purpose = 'Outdoor' THEN 'outdoor' WHEN texture = 'Plush' THEN 'plush' WHEN size ILIKE ('%9%x%12%') AND texture != 'Rubber' THEN '9x12' WHEN texture = 'Rubber' AND product_sub_type = 'classic' THEN 'classic_rug_pad' WHEN texture = 'Rubber' AND product_sub_type = 'cushioned' THEN 'cushioned_rug_pad' END product"

# product_categories_repl = "CASE WHEN shape = 'Rectangle' AND size NOT ILIKE ('%9%x%12%') AND texture = 'Chenille' AND purpose != 'Outdoor' THEN 'rectangular_chenille' WHEN texture = 'Shag' THEN 'shag' WHEN shape = 'Round' AND texture != 'Rubber' THEN 'round' WHEN purpose = 'Outdoor' THEN 'outdoor' WHEN texture = 'Plush' THEN 'plush' WHEN size ILIKE ('%9%x%12%') AND texture != 'Rubber' THEN '9x12' WHEN (texture = 'Rubber' AND product_sub_type = 'classic') OR (repl_texture = 'Rubber' AND repl_product_sub_type = 'classic') THEN 'classic_rug_pad' WHEN (texture = 'Rubber' AND product_sub_type = 'cushioned') OR (repl_texture = 'Rubber' AND repl_product_sub_type = 'cushioned') THEN 'cushioned_rug_pad' END product"

# Edit for cushion only non-cover.
product_cushioned = "CASE WHEN product_sub_type = 'cushioned' THEN 'cushioned' WHEN product_sub_type = 'classic' THEN 'classic' END product"

In [11]:
# Case for product sizes.
sizes = "CASE WHEN size ILIKE '3%x%5%' THEN '3x5' WHEN size ILIKE '8%r%' THEN '8-Round' WHEN size ILIKE '6%r%' THEN '6-Round' WHEN size ILIKE '2.5%x%10%' THEN '2.5x10' WHEN size ILIKE '9%x%12%' THEN '9x12' WHEN size ILIKE '8%x%10%' THEN '8x10' WHEN size ILIKE '5%x%7%' THEN '5x7' WHEN size ILIKE '2.5%x%7%' THEN '2.5x7' WHEN size ILIKE '6%x%9%' THEN '6x9' END size"

In [12]:
# Replacements primary queries.

# weekly_replacement_product = "SELECT a.look_forward_week_ended week_ended, a.view, a.product, a.plant, b.qty_replaced, a.qty_sold, b.qty_replaced * 1.0 / NULLIF(a.qty_sold, 0) rate FROM (SELECT CAST(DATE_TRUNC('week',fulfillment_date + %d) + INTERVAL '1 weeks' -1 AS DATE) look_forward_week_ended, %s, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a GROUP BY 1, 2, 3, 4) a LEFT JOIN (SELECT CAST(DATE_TRUNC('week',fulfillment_date + %d) + INTERVAL '1 weeks' -1 AS DATE) look_forward_week_ended, %s, SUM(CASE WHEN repl_created_date <= (fulfillment_date + %d) AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3, 4) b ON a.look_forward_week_ended = b.look_forward_week_ended AND a.product = b.product WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE)"

# monthly_replacement_product = "SELECT a.look_forward_month_ended month_ended, a.view, a.product, a.plant, b.qty_replaced, a.qty_sold, b.qty_replaced * 1.0 / NULLIF(a.qty_sold, 0) rate FROM (SELECT LAST_DAY(CAST(DATE_TRUNC('week',fulfillment_date + %d) + INTERVAL '1 weeks' -1 AS DATE)) look_forward_month_ended, %s, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final a GROUP BY 1, 2, 3, 4) a LEFT JOIN (SELECT LAST_DAY(CAST(DATE_TRUNC('week',fulfillment_date + %d) + INTERVAL '1 weeks' -1 AS DATE)) look_forward_month_ended, %s, SUM(CASE WHEN repl_created_date <= (fulfillment_date + %d) AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final GROUP BY 1, 2, 3, 4) b ON a.look_forward_month_ended = b.look_forward_month_ended AND a.product = b.product WHERE month_ended BETWEEN CAST(DATE_TRUNC('month',getdate()) - INTERVAL '13 month' -1 AS DATE) AND CAST(DATE_TRUNC('month',getdate()) + INTERVAL '1 month' -1 AS DATE)"

In [13]:
# Replacements
weekly_replacement_product_actual = "SELECT fulfillment_week_ended week_ended, %s, SUM(CASE WHEN repl_created_date IS NOT NULL AND time_to_repl IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) GROUP BY 1%s"

# weekly_replacement_product_forecast = "SELECT fulfillment_week_ended week_ended, %s, SUM(CASE WHEN repl_created_date IS NOT NULL AND time_to_repl IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold, CASE WHEN fulfillment_week_ended < '2020-11-01' THEN (qty_replaced * 1.0 / NULLIF(qty_sold, 0)) * 4.0 ELSE (qty_replaced * 1.0 / NULLIF(qty_sold, 0)) * 2.0 END rate FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) GROUP BY 1%s"

weekly_replacement_product_forecast = "SELECT fulfillment_week_ended + 21 week_ended, %s, CASE WHEN (SUM(CASE WHEN repl_created_date <= fulfillment_week_ended + 21 AND time_to_repl IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) * 2.2) > SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) THEN SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) ELSE SUM(CASE WHEN repl_created_date <= fulfillment_week_ended + 21 AND time_to_repl IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) * 2.2 END qty_replaced, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold, (qty_replaced * 1.0 / NULLIF(qty_sold, 0)) rate FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) GROUP BY 1%s"

weekly_replacement_product_time = "SELECT CAST(DATE_TRUNC('week',fulfillment_date + %d) + INTERVAL '1 weeks' -1 AS DATE) week_ended, %s, SUM(CASE WHEN time_to_repl IN %s AND plant IN %s THEN 1 ELSE 0 END) qty_replaced, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) GROUP BY 1%s"

monthly_replacement_product = "SELECT a.month_ended month_ended, CASE WHEN a.view = '_0_30' THEN '30_Day' WHEN a.view = '_31_90' THEN '90_Day' WHEN a.view = '_91_120' THEN '120_Day' WHEN a.view = '_121_240' THEN '240_Day' ELSE '240+_Day' END AS view, a.product product, a.plant plant, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, time_to_repl AS view%s, SUM(CASE WHEN repl_created_date IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) AND time_to_repl IS NOT NULL GROUP BY 1, 2%s) a LEFT JOIN(SELECT fulfillment_month_ended month_ended, 'actual' AS view%s, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) GROUP BY 1, 2%s) b ON a.month_ended = b.month_ended AND a.product = b.product"

monthly_replacement_product_actual = "SELECT a.month_ended month_ended, a.view AS view, a.product product, a.plant plant, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, 'actual' AS view%s, SUM(CASE WHEN repl_created_date IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) AND time_to_repl IS NOT NULL GROUP BY 1, 2%s) a LEFT JOIN(SELECT fulfillment_month_ended month_ended, 'actual' AS view%s, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) GROUP BY 1, 2%s) b ON a.month_ended = b.month_ended AND a.product = b.product"

monthly_replacement_product_size = "SELECT a.month_ended month_ended, CASE WHEN a.view = '_0_30' THEN '30_Day' WHEN a.view = '_31_90' THEN '90_Day' WHEN a.view = '_91_120' THEN '120_Day' WHEN a.view = '_121_240' THEN '240_Day' ELSE '240+_Day' END AS view, a.product product, a.plant plant, a.size size, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, time_to_repl AS view%s, SUM(CASE WHEN repl_created_date IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) AND time_to_repl IS NOT NULL GROUP BY 1, 2%s) a LEFT JOIN(SELECT fulfillment_month_ended month_ended, 'actual' AS view%s, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) GROUP BY 1, 2%s) b ON a.month_ended = b.month_ended AND a.product = b.product AND a.size = b.size"

monthly_replacement_product_size_actual = "SELECT a.month_ended month_ended, a.view AS view, a.product product, a.plant plant, a.size size, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, 'actual' AS view%s, SUM(CASE WHEN repl_created_date IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) AND time_to_repl IS NOT NULL GROUP BY 1, 2%s) a LEFT JOIN(SELECT fulfillment_month_ended month_ended, 'actual' AS view%s, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) GROUP BY 1, 2%s) b ON a.month_ended = b.month_ended AND a.product = b.product AND a.size = b.size"

In [14]:
# Replacements category
weekly_replacement_category_actual = "SELECT a.week_ended week_ended, a.view AS view, a.product product, a.plant plant, a.repl_category_dashboard master_category, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT fulfillment_week_ended week_ended, %s, repl_category_dashboard, SUM(CASE WHEN repl_created_date IS NOT NULL AND time_to_repl IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) AND repl_category_dashboard IS NOT NULL GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT fulfillment_week_ended week_ended, %s, SUM(CASE WHEN repl_created_date IS NOT NULL AND time_to_repl IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) AND repl_category_dashboard IS NOT NULL GROUP BY 1, 2, 3, 4) b ON a.week_ended = b.week_ended AND a.view = b.view AND a.product = b.product AND a.plant = b.plant"

weekly_replacement_category_time = "SELECT a.week_ended week_ended, a.view AS view, a.product product, a.plant plant, a.repl_category_dashboard master_category, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT CAST(DATE_TRUNC('week',fulfillment_date + %d) + INTERVAL '1 weeks' -1 AS DATE) week_ended, %s, repl_category_dashboard, SUM(CASE WHEN repl_created_date IS NOT NULL AND time_to_repl IN %s AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) AND repl_category_dashboard IS NOT NULL GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT CAST(DATE_TRUNC('week',fulfillment_date + %d) + INTERVAL '1 weeks' -1 AS DATE) week_ended, %s, SUM(CASE WHEN repl_created_date IS NOT NULL AND time_to_repl IN %s AND plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) AND repl_category_dashboard IS NOT NULL GROUP BY 1, 2, 3, 4) b ON a.week_ended = b.week_ended AND a.view = b.view AND a.product = b.product AND a.plant = b.plant"

weekly_replacement_category_forecast = "SELECT a.week_ended week_ended, a.view AS view, a.product product, a.plant plant, a.repl_category_dashboard master_category, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT fulfillment_week_ended + 21 week_ended, %s, repl_category_dashboard, SUM(CASE WHEN repl_created_date <= fulfillment_week_ended + 21 AND time_to_repl IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) AND repl_category_dashboard IS NOT NULL GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT fulfillment_week_ended + 21 week_ended, %s, SUM(CASE WHEN repl_created_date <= fulfillment_week_ended + 21 AND time_to_repl IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) AND repl_category_dashboard IS NOT NULL GROUP BY 1, 2, 3, 4) b ON a.week_ended = b.week_ended AND a.view = b.view AND a.product = b.product AND a.plant = b.plant"

monthly_replacement_category = "SELECT a.month_ended month_ended, CASE WHEN a.view = '_0_30' THEN '30_Day' WHEN a.view = '_31_90' THEN '90_Day' WHEN a.view = '_91_120' THEN '120_Day' WHEN a.view = '_121_240' THEN '240_Day' ELSE '240+_Day' END AS view, a.product product, a.plant plant, a.repl_category_dashboard master_category, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, time_to_repl AS view %s, repl_category_dashboard, SUM(CASE WHEN repl_created_date IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) AND time_to_repl IS NOT NULL AND repl_category_dashboard IS NOT NULL GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT fulfillment_month_ended month_ended, time_to_repl AS view %s, SUM(CASE WHEN repl_created_date IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) AND time_to_repl IS NOT NULL AND repl_category_dashboard IS NOT NULL GROUP BY 1, 2, 3, 4) b ON a.month_ended = b.month_ended AND a.view = b.view AND a.product = b.product AND a.plant = b.plant"

monthly_replacement_category_actual = "SELECT a.month_ended month_ended, a.view AS view, a.product product, a.plant plant, a.repl_category_dashboard master_category, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, 'actual' AS view %s, repl_category_dashboard, SUM(CASE WHEN repl_created_date IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) AND time_to_repl IS NOT NULL AND repl_category_dashboard IS NOT NULL GROUP BY 1, 2, 3, 4, 5) a LEFT JOIN (SELECT fulfillment_month_ended month_ended, 'actual' AS view %s, SUM(CASE WHEN repl_created_date IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) AND time_to_repl IS NOT NULL AND repl_category_dashboard IS NOT NULL GROUP BY 1, 2, 3, 4) b ON a.month_ended = b.month_ended AND a.view = b.view AND a.product = b.product AND a.plant = b.plant"

In [15]:
# Replacement reasons.
weekly_replacement_reason_actual = "SELECT a.week_ended week_ended, a.view AS view, a.product product, a.size size, a.plant plant, a.repl_reason master_reason, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT fulfillment_week_ended week_ended, %s, repl_reason, SUM(CASE WHEN repl_created_date IS NOT NULL AND time_to_repl IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) AND repl_reason IS NOT NULL GROUP BY 1, 2, 3, 4, 5, 6) a LEFT JOIN (SELECT fulfillment_week_ended week_ended, %s, SUM(CASE WHEN repl_created_date IS NOT NULL AND time_to_repl IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) AND repl_reason IS NOT NULL GROUP BY 1, 2, 3, 4, 5) b ON a.week_ended = b.week_ended AND a.view = b.view AND a.product = b.product AND a.size = b.size AND a.plant = b.plant"

weekly_replacement_reason_time = "SELECT a.week_ended week_ended, a.view AS view, a.product product, a.size size, a.plant plant, a.repl_reason master_reason, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT CAST(DATE_TRUNC('week',fulfillment_date + %d) + INTERVAL '1 weeks' -1 AS DATE) week_ended, %s, repl_reason, SUM(CASE WHEN repl_created_date IS NOT NULL AND time_to_repl IN %s AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) AND repl_reason IS NOT NULL GROUP BY 1, 2, 3, 4, 5, 6) a LEFT JOIN (SELECT CAST(DATE_TRUNC('week',fulfillment_date + %d) + INTERVAL '1 weeks' -1 AS DATE) week_ended, %s, SUM(CASE WHEN repl_created_date IS NOT NULL AND time_to_repl IN %s AND plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) AND repl_reason IS NOT NULL GROUP BY 1, 2, 3, 4, 5) b ON a.week_ended = b.week_ended AND a.view = b.view AND a.product = b.product AND a.size = b.size AND a.plant = b.plant"

weekly_replacement_reason_forecast = "SELECT a.week_ended week_ended, a.view AS view, a.product product, a.size size, a.plant plant, a.repl_reason master_reason, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT fulfillment_week_ended + 21 week_ended, %s, repl_reason, SUM(CASE WHEN repl_created_date <= fulfillment_week_ended + 21 AND time_to_repl IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) AND repl_reason IS NOT NULL GROUP BY 1, 2, 3, 4, 5, 6) a LEFT JOIN (SELECT fulfillment_week_ended + 21 week_ended, %s, SUM(CASE WHEN repl_created_date <= fulfillment_week_ended + 21 AND time_to_repl IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE week_ended BETWEEN CAST(DATE_TRUNC('week',getdate()) - INTERVAL '56 weeks' -1 AS DATE) AND CAST(DATE_TRUNC('week',getdate()) + INTERVAL '1 weeks' -1 AS DATE) AND repl_reason IS NOT NULL GROUP BY 1, 2, 3, 4, 5) b ON a.week_ended = b.week_ended AND a.view = b.view AND a.product = b.product AND a.size = b.size AND a.plant = b.plant"

monthly_replacement_reason = "SELECT a.month_ended month_ended, CASE WHEN a.view = '_0_30' THEN '30_Day' WHEN a.view = '_31_90' THEN '90_Day' WHEN a.view = '_91_120' THEN '120_Day' WHEN a.view = '_121_240' THEN '240_Day' ELSE '240+_Day' END AS view, a.product product, a.size size, a.plant plant, a.repl_reason master_reason, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, time_to_repl AS view %s, repl_reason, SUM(CASE WHEN repl_created_date IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) AND time_to_repl IS NOT NULL AND repl_reason IS NOT NULL GROUP BY 1, 2, 3, 4, 5, 6) a LEFT JOIN (SELECT fulfillment_month_ended month_ended, time_to_repl AS view %s, SUM(CASE WHEN repl_created_date IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) AND time_to_repl IS NOT NULL AND repl_reason IS NOT NULL GROUP BY 1, 2, 3, 4, 5) b ON a.month_ended = b.month_ended AND a.view = b.view AND a.product = b.product AND a.size = b.size AND a.plant = b.plant"

monthly_replacement_reason_actual = "SELECT a.month_ended month_ended, a.view AS view, a.product product, a.size size, a.plant plant, a.repl_reason master_reason, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, 'actual' AS view %s, repl_reason, SUM(CASE WHEN repl_created_date IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) AND time_to_repl IS NOT NULL AND repl_reason IS NOT NULL GROUP BY 1, 2, 3, 4, 5, 6) a LEFT JOIN (SELECT fulfillment_month_ended month_ended, 'actual' AS view %s, SUM(CASE WHEN repl_created_date IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) AND time_to_repl IS NOT NULL AND repl_reason IS NOT NULL GROUP BY 1, 2, 3, 4, 5) b ON a.month_ended = b.month_ended AND a.view = b.view AND a.product = b.product AND a.size = b.size AND a.plant = b.plant"

In [16]:
# Replacements style.
monthly_replacement_style = "SELECT a.month_ended month_ended, CASE WHEN a.view = '_0_30' THEN '30_Day' WHEN a.view = '_31_90' THEN '90_Day' WHEN a.view = '_91_120' THEN '120_Day' WHEN a.view = '_121_240' THEN '240_Day' ELSE '240+_Day' END AS view, a.rug_name rug_name, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, time_to_repl AS view%s, SUM(CASE WHEN repl_created_date IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) AND time_to_repl IS NOT NULL GROUP BY 1, 2%s) a LEFT JOIN(SELECT fulfillment_month_ended month_ended, 'actual' AS view%s, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) GROUP BY 1, 2%s) b ON a.month_ended = b.month_ended AND a.rug_name = b.rug_name"

monthly_replacement_style_actual = "SELECT a.month_ended month_ended, a.view AS view, a.rug_name rug_name, a.qty_replaced qty_replaced, b.qty_sold qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM (SELECT fulfillment_month_ended month_ended, 'actual' AS view%s, SUM(CASE WHEN repl_created_date IS NOT NULL AND plant IN %s THEN 1 ELSE 0 END) qty_replaced FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) AND time_to_repl IS NOT NULL GROUP BY 1, 2%s) a LEFT JOIN(SELECT fulfillment_month_ended month_ended, 'actual' AS view%s, SUM(CASE WHEN plant IN %s THEN 1 ELSE 0 END) qty_sold FROM product_pull.ret_exch_rep_data_final WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) GROUP BY 1, 2%s) b ON a.month_ended = b.month_ended AND a.rug_name = b.rug_name"

In [17]:
# # Weekly recovery
# weekly_recovery_product_actual = "SELECT fulfillment_week_ended week_ended, 'actual' AS view%s, SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND %s THEN 1 ELSE 0 END) qty_replaced, SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s THEN 1 ELSE 0 END) qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM product_pull.ret_exch_rep_data_final a WHERE week_ended BETWEEN CAST(DATE_TRUNC('week', getdate()) - INTERVAL '56 weeks' - 1 AS DATE) AND CAST(DATE_TRUNC('week', getdate()) + INTERVAL '1 weeks' - 1 AS DATE) GROUP BY 1%s"

# # Monthly recovery
# monthly_recovery_product_actual = "SELECT fulfillment_month_ended month_ended, 'actual' AS view%s, SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND %s THEN 1 ELSE 0 END) qty_replaced, SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s THEN 1 ELSE 0 END) qty_sold, qty_replaced * 1.0 / NULLIF(qty_sold, 0) rate FROM product_pull.ret_exch_rep_data_final a WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE) AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE) GROUP BY 1%s"

In [18]:
product_categories_overall = "CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12') THEN 'overall_cover' ELSE 'overall_pad' END product"

In [19]:
# Weekly Recovery
weekly_recovery_product_breakdown = """SELECT CAST(DATE_TRUNC('week', return_receive_date) + INTERVAL '1 weeks' - 1 AS DATE) week_ended,
 'actual' AS view%s
 , CASE
 WHEN product IN ('rectangular_chenille',  'shag', 'round', 'outdoor', 'plush', '9x12')
 THEN SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s
 AND (cover_recovery_status = 'Add to Inventory') THEN 1 ELSE 0 END)
 WHEN product IN ('classic_rug_pad', 'cushioned_rug_pad')
 THEN SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s
 AND (pad_recovery_status = 'Add to Inventory') THEN 1 ELSE 0 END)
 END qty_recovered,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND (CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status IS NOT NULL WHEN product_categories IN ('classic_rug_pad', 'cushioned_rug_pad', 'overall_pad') THEN pad_recovery_status IS NOT NULL END) THEN 1 ELSE 0 END) qty_sold,
 qty_recovered * 1.0 / NULLIF(qty_sold, 0) rate
 FROM product_pull.ret_exch_rep_data_final a
 WHERE week_ended BETWEEN CAST(DATE_TRUNC('week', getdate()) - INTERVAL '56 weeks' - 1 AS DATE)
 AND CAST(DATE_TRUNC('week', getdate()) + INTERVAL '1 weeks' - 1 AS DATE)
 GROUP BY 1%s"""

weekly_recovery_product_breakdown_overall = """SELECT CAST(DATE_TRUNC('week', return_receive_date) + INTERVAL '1 weeks' - 1 AS DATE) week_ended,
 'actual' AS view%s
 , CASE
 WHEN product IN ('overall_cover')
 THEN SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s
 AND (cover_recovery_status = 'Add to Inventory') THEN 1 ELSE 0 END)
 WHEN product IN ('overall_pad')
 THEN SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s
 AND (pad_recovery_status = 'Add to Inventory') THEN 1 ELSE 0 END)
 END qty_recovered,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND (CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status IS NOT NULL WHEN product_categories IN ('classic_rug_pad', 'cushioned_rug_pad', 'overall_pad') THEN pad_recovery_status IS NOT NULL END) THEN 1 ELSE 0 END) qty_sold,
 qty_recovered * 1.0 / NULLIF(qty_sold, 0) rate
 FROM product_pull.ret_exch_rep_data_final a
 WHERE week_ended BETWEEN CAST(DATE_TRUNC('week', getdate()) - INTERVAL '56 weeks' - 1 AS DATE)
 AND CAST(DATE_TRUNC('week', getdate()) + INTERVAL '1 weeks' - 1 AS DATE)
 GROUP BY 1%s"""

weekly_recovery_product_overall = """SELECT CAST(DATE_TRUNC('week', return_receive_date) + INTERVAL '1 weeks' - 1 AS DATE) week_ended,
 'actual' AS view%s,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s
 AND (cover_recovery_status = 'Add to Inventory' OR pad_recovery_status = 'Add to Inventory') THEN 1 ELSE 0 END) qty_recovered,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND (CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status IS NOT NULL WHEN product_categories IN ('classic_rug_pad', 'cushioned_rug_pad', 'overall_pad') THEN pad_recovery_status IS NOT NULL END) THEN 1 ELSE 0 END) qty_sold,
 qty_recovered * 1.0 / NULLIF(qty_sold, 0) rate
 FROM product_pull.ret_exch_rep_data_final a
 WHERE week_ended BETWEEN CAST(DATE_TRUNC('week', getdate()) - INTERVAL '56 weeks' - 1 AS DATE)
 AND CAST(DATE_TRUNC('week', getdate()) + INTERVAL '1 weeks' - 1 AS DATE)
 GROUP BY 1%s"""

# Monthly Recovery
monthly_recovery_product_breakdown = """SELECT LAST_DAY(return_receive_date) month_ended,
 'actual' AS view%s
 , CASE
 WHEN product IN ('rectangular_chenille',  'shag', 'round', 'outdoor', 'plush', '9x12')
 THEN SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s
 AND (cover_recovery_status = 'Add to Inventory') THEN 1 ELSE 0 END)
 WHEN product IN ('classic_rug_pad', 'cushioned_rug_pad')
 THEN SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s
 AND (pad_recovery_status = 'Add to Inventory') THEN 1 ELSE 0 END)
 END qty_recovered,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND (CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status IS NOT NULL WHEN product_categories IN ('classic_rug_pad', 'cushioned_rug_pad', 'overall_pad') THEN pad_recovery_status IS NOT NULL END) THEN 1 ELSE 0 END) qty_sold,
 qty_recovered * 1.0 / NULLIF(qty_sold, 0) rate
 FROM product_pull.ret_exch_rep_data_final a
 WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE)
 AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE)
 GROUP BY 1%s"""

monthly_recovery_product_breakdown_overall = """SELECT LAST_DAY(return_receive_date) month_ended,
 'actual' AS view%s
 , CASE
 WHEN product IN ('overall_cover')
 THEN SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s
 AND (cover_recovery_status = 'Add to Inventory') THEN 1 ELSE 0 END)
 WHEN product IN ('overall_pad')
 THEN SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s
 AND (pad_recovery_status = 'Add to Inventory') THEN 1 ELSE 0 END)
 END qty_recovered,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND (CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status IS NOT NULL WHEN product_categories IN ('classic_rug_pad', 'cushioned_rug_pad', 'overall_pad') THEN pad_recovery_status IS NOT NULL END) THEN 1 ELSE 0 END) qty_sold,
 qty_recovered * 1.0 / NULLIF(qty_sold, 0) rate
 FROM product_pull.ret_exch_rep_data_final a
 WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE)
 AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE)
 GROUP BY 1%s"""

monthly_recovery_product_overall = """SELECT LAST_DAY(return_receive_date) month_ended,
 'actual' AS view%s,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s
 AND (cover_recovery_status = 'Add to Inventory' OR pad_recovery_status = 'Add to Inventory') THEN 1 ELSE 0 END) qty_recovered,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND (CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status IS NOT NULL WHEN product_categories IN ('classic_rug_pad', 'cushioned_rug_pad', 'overall_pad') THEN pad_recovery_status IS NOT NULL END) THEN 1 ELSE 0 END) qty_sold,
 qty_recovered * 1.0 / NULLIF(qty_sold, 0) rate
 FROM product_pull.ret_exch_rep_data_final a
 WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE)
 AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE)
 GROUP BY 1%s"""

In [20]:
weekly_recovery_product_category_breakdown = """SELECT
 a.week_ended week_ended,
 a.view AS view,
 a.product product,
 a.send_back__type send_back__type,
 a.plant plant,
 a.master_category master_category,
 a.qty_recovered qty_recovered,
 b.qty_sold qty_sold,
 qty_recovered * 1.0 / NULLIF(qty_sold, 0) rate
 FROM (
 SELECT
 CAST(DATE_TRUNC('week', return_receive_date) + INTERVAL '1 weeks' - 1 AS DATE) week_ended,
 'actual' AS view,
 product_categories product%s,
 CASE
 WHEN product IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status
 ELSE pad_recovery_status
 END master_category,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND (CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status IS NOT NULL WHEN product_categories IN ('classic_rug_pad', 'cushioned_rug_pad', 'overall_pad') THEN pad_recovery_status IS NOT NULL END) THEN 1 ELSE 0 END) qty_recovered
 FROM product_pull.ret_exch_rep_data_final a
 WHERE week_ended BETWEEN CAST(DATE_TRUNC('week', getdate()) - INTERVAL '56 weeks' - 1 AS DATE)
 AND CAST(DATE_TRUNC('week', getdate()) + INTERVAL '1 weeks' - 1 AS DATE)
 GROUP BY 1, 2, 3, 4, 5, 6
 ) a
 LEFT JOIN (
 SELECT
 CAST(DATE_TRUNC('week', return_receive_date) + INTERVAL '1 weeks' - 1 AS DATE) week_ended,
 'actual' AS view,
 product_categories product%s,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND (CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status IS NOT NULL WHEN product_categories IN ('classic_rug_pad', 'cushioned_rug_pad', 'overall_pad') THEN pad_recovery_status IS NOT NULL END) THEN 1 ELSE 0 END) qty_sold
 FROM product_pull.ret_exch_rep_data_final a
 WHERE week_ended BETWEEN CAST(DATE_TRUNC('week', getdate()) - INTERVAL '56 weeks' - 1 AS DATE)
 AND CAST(DATE_TRUNC('week', getdate()) + INTERVAL '1 weeks' - 1 AS DATE)
 GROUP BY 1, 2, 3, 4, 5
 ) b
 ON a.week_ended = b.week_ended
 AND a.view = b.view
 AND a.product = b.product
 AND a.send_back__type = b.send_back__type
 AND a.plant = b.plant
 WHERE master_category IS NOT NULL"""

weekly_recovery_product_category_breakdown_overall = """SELECT
 a.week_ended week_ended,
 a.view AS view,
 a.product product,
 a.send_back__type send_back__type,
 a.plant plant,
 a.master_category master_category,
 a.qty_recovered qty_recovered,
 b.qty_sold qty_sold,
 qty_recovered * 1.0 / NULLIF(qty_sold, 0) rate
 FROM (
 SELECT
 CAST(DATE_TRUNC('week', return_receive_date) + INTERVAL '1 weeks' - 1 AS DATE) week_ended,
 'actual' AS view,
 CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12') THEN 'overall_cover' ELSE 'overall_pad' END product%s,
 CASE
 WHEN product IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status
 ELSE pad_recovery_status
 END master_category,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND (CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status IS NOT NULL WHEN product_categories IN ('classic_rug_pad', 'cushioned_rug_pad', 'overall_pad') THEN pad_recovery_status IS NOT NULL END) THEN 1 ELSE 0 END) qty_recovered
 FROM product_pull.ret_exch_rep_data_final a
 WHERE week_ended BETWEEN CAST(DATE_TRUNC('week', getdate()) - INTERVAL '56 weeks' - 1 AS DATE)
 AND CAST(DATE_TRUNC('week', getdate()) + INTERVAL '1 weeks' - 1 AS DATE)
 GROUP BY 1, 2, 3, 4, 5, 6
 ) a
 LEFT JOIN (
 SELECT
 CAST(DATE_TRUNC('week', return_receive_date) + INTERVAL '1 weeks' - 1 AS DATE) week_ended,
 'actual' AS view,
 CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12') THEN 'overall_cover' ELSE 'overall_pad' END product%s,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND (CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status IS NOT NULL WHEN product_categories IN ('classic_rug_pad', 'cushioned_rug_pad', 'overall_pad') THEN pad_recovery_status IS NOT NULL END) THEN 1 ELSE 0 END) qty_sold
 FROM product_pull.ret_exch_rep_data_final a
 WHERE week_ended BETWEEN CAST(DATE_TRUNC('week', getdate()) - INTERVAL '56 weeks' - 1 AS DATE)
 AND CAST(DATE_TRUNC('week', getdate()) + INTERVAL '1 weeks' - 1 AS DATE)
 GROUP BY 1, 2, 3, 4, 5
 ) b
 ON a.week_ended = b.week_ended
 AND a.view = b.view
 AND a.product = b.product
 AND a.send_back__type = b.send_back__type
 AND a.plant = b.plant
 WHERE master_category IS NOT NULL"""


monthly_recovery_product_category_breakdown = """SELECT
 a.month_ended month_ended,
 a.view AS view,
 a.product product,
 a.send_back__type send_back__type,
 a.plant plant,
 a.master_category master_category,
 a.qty_recovered qty_recovered,
 b.qty_sold qty_sold,
 qty_recovered * 1.0 / NULLIF(qty_sold, 0) rate
 FROM (
 SELECT
 LAST_DAY(return_receive_date) month_ended,
 'actual' AS view,
 product_categories product%s,
 CASE
 WHEN product IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status
 ELSE pad_recovery_status
 END master_category,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND (CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status IS NOT NULL WHEN product_categories IN ('classic_rug_pad', 'cushioned_rug_pad', 'overall_pad') THEN pad_recovery_status IS NOT NULL END) THEN 1 ELSE 0 END) qty_recovered
 FROM product_pull.ret_exch_rep_data_final a
 WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE)
 AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE)
 GROUP BY 1, 2, 3, 4, 5, 6
 ) a
 LEFT JOIN (
 SELECT
 LAST_DAY(return_receive_date) month_ended,
 'actual' AS view,
 product_categories product%s,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND (CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status IS NOT NULL WHEN product_categories IN ('classic_rug_pad', 'cushioned_rug_pad', 'overall_pad') THEN pad_recovery_status IS NOT NULL END) THEN 1 ELSE 0 END) qty_sold
 FROM product_pull.ret_exch_rep_data_final a
 WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE)
 AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE)
 GROUP BY 1, 2, 3, 4, 5
 ) b
 ON a.month_ended = b.month_ended
 AND a.view = b.view
 AND a.product = b.product
 AND a.send_back__type = b.send_back__type
 AND a.plant = b.plant
 WHERE master_category IS NOT NULL"""

monthly_recovery_product_category_breakdown_overall = """SELECT
 a.month_ended month_ended,
 a.view AS view,
 a.product product,
 a.send_back__type send_back__type,
 a.plant plant,
 a.master_category master_category,
 a.qty_recovered qty_recovered,
 b.qty_sold qty_sold,
 qty_recovered * 1.0 / NULLIF(qty_sold, 0) rate
 FROM (
 SELECT
 LAST_DAY(return_receive_date) month_ended,
 'actual' AS view,
 CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12') THEN 'overall_cover' ELSE 'overall_pad' END product%s,
 CASE
 WHEN product IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status
 ELSE pad_recovery_status
 END master_category,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND (CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status IS NOT NULL WHEN product_categories IN ('classic_rug_pad', 'cushioned_rug_pad', 'overall_pad') THEN pad_recovery_status IS NOT NULL END) THEN 1 ELSE 0 END) qty_recovered
 FROM product_pull.ret_exch_rep_data_final a
 WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE)
 AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE)
 GROUP BY 1, 2, 3, 4, 5, 6
 ) a
 LEFT JOIN (
 SELECT
 LAST_DAY(return_receive_date) month_ended,
 'actual' AS view,
 CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12') THEN 'overall_cover' ELSE 'overall_pad' END product%s,
 SUM(CASE WHEN return_receive_date IS NOT NULL AND send_back__type IN %s AND plant IN %s AND (CASE WHEN product_categories IN ('rectangular_chenille', 'shag', 'round', 'outdoor', 'plush', '9x12', 'overall_cover') THEN cover_recovery_status IS NOT NULL WHEN product_categories IN ('classic_rug_pad', 'cushioned_rug_pad', 'overall_pad') THEN pad_recovery_status IS NOT NULL END) THEN 1 ELSE 0 END) qty_sold
 FROM product_pull.ret_exch_rep_data_final a
 WHERE month_ended BETWEEN CAST(DATE_TRUNC('month', getdate()) - INTERVAL '13 month' - 1 AS DATE)
 AND CAST(DATE_TRUNC('month', getdate()) + INTERVAL '1 month' - 1 AS DATE)
 GROUP BY 1, 2, 3, 4, 5
 ) b
 ON a.month_ended = b.month_ended
 AND a.view = b.view
 AND a.product = b.product
 AND a.send_back__type = b.send_back__type
 AND a.plant = b.plant
 WHERE master_category IS NOT NULL"""

In [21]:
# # Recovery type conditions.
# recovery_type_both = "(cover_recovery_status = 'Add to Inventory' AND pad_recovery_status = 'Add to Inventory')"
# recovery_type_cover = "(cover_recovery_status = 'Add to Inventory' AND (pad_recovery_status != 'Add to Inventory' OR pad_recovery_status IS NULL))"
# recovery_type_pad = "(pad_recovery_status = 'Add to Inventory' AND (cover_recovery_status != 'Add to Inventory' OR cover_recovery_status IS NULL))"

In [26]:
# Query paramters master dictionary.

# Update paramters as needed.
query_params = {
    'send_back_product_weekly': {'view': {'actual': [weekly_actual_query, actual_qty], 'forecast': [weekly_forecast_query, weekly_forecast_qty]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}},
    'send_back_product_monthly': {'view': {'actual': [monthly_actual_query, actual_qty]},
                          'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                          'send_back__type': {'send_back_both': "('return', 'exchange')",
                                              'return': "('return')", 'exchange': "('exchange')"},
                            'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}},
    'send_back_product_size_weekly': {'view': {'actual': [weekly_actual_query, actual_qty], 'forecast': [weekly_forecast_query, weekly_forecast_qty]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"},
                                'size': {'all': "'all' size", 'size_categories': sizes}},
    'send_back_product_size_monthly': {'view': {'actual': [monthly_actual_query, actual_qty]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"},
                                'size': {'all': "'all' size", 'size_categories': sizes}},
    'send_back_product_category_weekly': {'view': {'actual': weekly_actual_category_query, 'forecast': weekly_forecast_category_query},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}
                                         },
    'send_back_product_category_monthly': {'view': {'actual': monthly_actual_category_query},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}
                                          },
    'send_back_product_size_reason_weekly': {'view': {'actual': weekly_actual_reason_query, 'forecast': weekly_forecast_reason_query},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'size': {'all': "'all' size", 'size_categories': sizes}
                                            },
    'send_back_product_size_reason_monthly': {'view': {'actual': monthly_actual_reason_query},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'size': {'all': "'all' size", 'size_categories': sizes}
                                             },
    'send_back_style_weekly': {'view': {'actual': [weekly_actual_query, actual_qty_style], 'forecast': [weekly_forecast_query, weekly_forecast_qty_style]},
                              'rug_name': {'rug_name': " design_name rug_name"},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"}},
    'send_back_style_monthly': {'view': {'actual': [monthly_actual_query, actual_qty_style]},
                              'rug_name': {'rug_name': " design_name rug_name"},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"}},
    'replacement_product_weekly': {'view': {'actual': [weekly_replacement_product_actual, 'actual', "-"], 'forecast': [weekly_replacement_product_forecast, 'forecast', "-"]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}},
    'replacement_product_monthly': {'view': {'time_to_repl': [monthly_replacement_product], 'actual': [monthly_replacement_product_actual]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}},
    'replacement_product_size_weekly': {'view': {'actual': [weekly_replacement_product_actual, 'actual', "-"], 'forecast': [weekly_replacement_product_forecast, 'forecast', "-"]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"},
                                'size': {'all': "'all' size", 'size_categories': sizes}},
    'replacement_product_size_monthly': {'view': {'time_to_repl': [monthly_replacement_product_size], 'actual': [monthly_replacement_product_size_actual]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"},
                                'size': {'all': "'all' size", 'size_categories': sizes}},
    'replacement_product_size_category_weekly': {'view': {'actual': [weekly_replacement_category_actual, 'actual', "-"], 'forecast': [weekly_replacement_category_forecast, 'forecast', '-']},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}
                                         },
    'replacement_product_size_category_monthly': {'view': {'time_to_repl': [monthly_replacement_category], 'actual': [monthly_replacement_category_actual]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}},
    'replacement_product_size_reason_weekly': {'view': {'actual': [weekly_replacement_reason_actual, 'actual', "-"], 'forecast': [weekly_replacement_reason_forecast, 'forecast', '-']},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"},
                                'size': {'all': "'all' size", 'size_categories': sizes}},
    'replacement_product_size_reason_monthly': {'view': {'time_to_repl': [monthly_replacement_reason], 'actual': [monthly_replacement_reason_actual]},
                              'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
                              'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"},
                                'size': {'all': "'all' size", 'size_categories': sizes}},
    'replacement_style_weekly': {'view': {'actual': [weekly_replacement_product_actual, 'actual', "-"], 'forecast': [weekly_replacement_product_forecast, 'forecast', "-"]},
                                   'rug_name': {'rug_name': " design_name rug_name"},
                                   'plant': {'all': "('LA', 'CHI', 'N/A')"}},
    'replacement_style_monthly': {'view': {'time_to_repl': [monthly_replacement_style], 'actual': [monthly_replacement_style_actual]},
                              'rug_name': {'rug_name': " design_name rug_name"},
                              'plant': {'all': "('LA', 'CHI', 'N/A')"}},
    'recovery_product_weekly': {'view': {'product_overall': [weekly_recovery_product_overall.replace('\n', '')],
                                         'product_breakdown': [weekly_recovery_product_breakdown.replace('\n', '')],
                                         'product_breakdown_overall': [weekly_recovery_product_breakdown_overall.replace('\n', '')]},
                                'product': {'all': "'all' product", 'categories': 'product_categories product', 'categories_overall': product_categories_overall},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}},
    'recovery_product_monthly': {'view': {'product_overall': [monthly_recovery_product_overall.replace('\n', '')],
                                         'product_breakdown': [monthly_recovery_product_breakdown.replace('\n', '')],
                                         'product_breakdown_overall': [monthly_recovery_product_breakdown_overall.replace('\n', '')]},
                                'product': {'all': "'all' product", 'categories': 'product_categories product', 'categories_overall': product_categories_overall},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}},
    'recovery_product_size_weekly': {'view': {'product_overall': [weekly_recovery_product_overall.replace('\n', '')],
                                         'product_breakdown': [weekly_recovery_product_breakdown.replace('\n', '')],
                                         'product_breakdown_overall': [weekly_recovery_product_breakdown_overall.replace('\n', '')]},
                                'product': {'all': "'all' product", 'categories': 'product_categories product', 'categories_overall': product_categories_overall},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"},
                                'size': {'all': "'all' size", 'size_categories': sizes}},
    'recovery_product_size_monthly': {'view': {'product_overall': [monthly_recovery_product_overall.replace('\n', '')],
                                         'product_breakdown': [monthly_recovery_product_breakdown.replace('\n', '')],
                                         'product_breakdown_overall': [monthly_recovery_product_breakdown_overall.replace('\n', '')]},
                                'product': {'all': "'all' product", 'categories': 'product_categories product', 'categories_overall': product_categories_overall},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"},
                                'size': {'all': "'all' size", 'size_categories': sizes}},
    'recovery_product_category_weekly': {'view': {'product_breakdown': [weekly_recovery_product_category_breakdown.replace('\n', '')],
                                         'product_breakdown_overall': [weekly_recovery_product_category_breakdown_overall.replace('\n', '')]},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}},
    'recovery_product_category_monthly': {'view': {'product_breakdown': [monthly_recovery_product_category_breakdown.replace('\n', '')],
                                         'product_breakdown_overall': [monthly_recovery_product_category_breakdown_overall.replace('\n', '')]},
                              'send_back__type': {'send_back_both': "('return', 'exchange')",
                                                  'return': "('return')", 'exchange': "('exchange')"},
                                'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}}
}

In [27]:
# Creates unioned sql for each tab in workbook based on query parameters.
for key, value in query_params.items():
    print(key)
    # Create/update empty DF.
    tab_df = pd.DataFrame()
    # Create/update empty string for master query.
    master_query = ""
    # Create list to hold all possible combinations per tab.
    combinations = []
    # Same as above, used to get total # of combinations.
    check = []
    # Get all dict keys.
    keys = value.keys()
    # Create tuple for all possible params per query.
    values = (value[key] for key in keys)
    # Build generator that will be unpacked as we define query.
    combinations.append(dict(zip(keys, combination)) for combination in itertools.product(*values))
    # Same as above, duplicated for use in total # of combinations.
    values2 = (value[key] for key in keys)
    check.append(dict(zip(keys, combination)) for combination in itertools.product(*values2))
    for c in check:
        # Total # of combinations calculation.
        total = len(list(c))
    # Iterate through all possible combinations and enumerate to access current i.
    for combination in combinations:
        for item, combo in enumerate(combination):
            # Define list of all required columns.
            keys = list(keys)
            # Create empty string to store group by statement.
            grouping = ""
            params = ""
            if key in ('recovery_product_weekly', 'recovery_product_monthly', 'recovery_product_size_weekly', 'recovery_product_size_monthly'):
                if (combo['view'] == 'product_breakdown' and combo['product'] == 'categories_overall') or (combo['view'] == 'product_breakdown_overall' and combo['product'] == 'categories'):
    #                 print(combo)
                    continue
                if (combo['view'] == 'product_overall' and combo['product'] == 'categories') or (combo['view'] == 'product_overall' and combo['product'] == 'categories_overall'):
                    continue
            # Iterate through all keys using index to access key variables.
            for i in range(len(keys)):
#                 print(query_params[key][keys[i]][combo[keys[i]]])
                if 'replacement' in key:
                    if 'monthly' in key:
                        if i == 0:
                            query = query_params[key][keys[i]][combo[keys[i]]][0]
#                             print(query)
                        elif keys[i] == 'plant':
                            plant = str(query_params[key][keys[i]][combo[keys[i]]])
                            if key != 'replacement_style_monthly':
                                grouping = f"{grouping}, {i+2}"
                                params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
#                             print(params)
                        else:
                            grouping = f"{grouping}, {i+2}"
                            params = f"{params}, {query_params[key][keys[i]][combo[keys[i]]]}"
#                             print(params)
                    else:
                        # Continuously add column index to group by statement.
                        grouping = f"{grouping}, {i+2}"
                        if i == 0:
                            query = query_params[key][keys[i]][combo[keys[i]]][0]
                            params = f"'{combo[keys[i]]}' AS {keys[i]}"
                            time_interval = query_params[key][keys[i]][combo[keys[i]]][1]
                            repl_time = query_params[key][keys[i]][combo[keys[i]]][2]
                        elif keys[i] == 'plant':
                            plant = str(query_params[key][keys[i]][combo[keys[i]]])
                            params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
                        else:
                            params = f"{params}, {query_params[key][keys[i]][combo[keys[i]]]}"
                # Logic for recovery.
                elif 'recovery' in key:
#                     print(combo['view'])
                    # Continuously add column index to group by statement.
                    grouping = f"{grouping}, {i+2}"
                    if i == 0:
                        query = query_params[key][keys[i]][combo[keys[i]]][0]
#                         print(query)
                    # Store params for send_back__type and plant, and store in separate variables: to be added to calculation.
                    elif keys[i] == 'send_back__type' or keys[i] == 'plant':
                        if keys[i] == 'send_back__type':
                            send_back_type = str(query_params[key][keys[i]][combo[keys[i]]])
                            params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
#                             print(params)
                        else:
                            plant = str(query_params[key][keys[i]][combo[keys[i]]])
                            params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
#                             print(plant)
#                     elif keys[i] == 'recovery_type':
#                         recovery_logic = str(query_params[key][keys[i]][combo[keys[i]]])
#                         params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
                    # Otherwise, add to params.
                    else:
                        params = f"{params}, {query_params[key][keys[i]][combo[keys[i]]]}"
                # Proceed with logic if building query for either category, or reeason as sql logic differs from others.
                elif 'category' in key or 'reason' in key:
                    # Locate the first column, "view".
                    if i == 0:
                        # Init query and params.
                        query = query_params[key][keys[i]][combo[keys[i]]]
                        params = f"'{combo[keys[i]]}' AS {keys[i]}"
                    # Store params for send_back__type and plant, and store in separate variables: to be added to calculation.
                    elif keys[i] == 'send_back__type' or keys[i] == 'plant':
                        if keys[i] == 'send_back__type':
                            send_back_type = str(query_params[key][keys[i]][combo[keys[i]]])
                        else:
                            plant = str(query_params[key][keys[i]][combo[keys[i]]])
                        # Add params to both regardless.
                        params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
                    # Otherwise, add to params.
                    else:
                        params = f"{params}, {query_params[key][keys[i]][combo[keys[i]]]}"
                # For all other tabs...
                else:
                    # Continuously add column index to group by statement.
                    grouping = f"{grouping}, {i+2}"
                    # For first column, "view"
                    if i == 0:
                        # Init query and params, and store calculation in variable.
                        query = query_params[key][keys[i]][combo[keys[i]]][0]
                        calculation = query_params[key][keys[i]][combo[keys[i]]][1]
                        params = f"'{combo[keys[i]]}' AS {keys[i]}"
                    # For all other columns...
                    else:
                        # Determine if current iteration for send_back__type.
                        if keys[i] == 'send_back__type':
                            # Proceed only for style tabs.
                            if 'style' in key:
                                # Store send_back__type, add to caclulation as only variable passed in, and add to params.
                                send_back_type = str(query_params[key][keys[i]][combo[keys[i]]])
                                calculation = calculation %(send_back_type)
                                params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
                            # Otherwise, grab send_back__type and add to params.
                            else:
                                send_back_type = str(query_params[key][keys[i]][combo[keys[i]]])
                                params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
                        # Determine if current iteration for plant.
                        elif keys[i] == 'plant':
                            # Store send_back__type and plant in calculation and add to params.
                            plant = str(query_params[key][keys[i]][combo[keys[i]]])
                            calculation = calculation %(send_back_type, plant)
                            params = f"{params}, '{combo[keys[i]]}' {keys[i]}"
                        else:
                            # For all others, add to params.
                            params = f"{params}, {query_params[key][keys[i]][combo[keys[i]]]}"
            # Once all params are collected and variables are defined, build final query.
            if 'replacement' in key:
                if 'product_weekly' in key or 'product_size_weekly' in key or 'style_weekly' in key:
                    if 'actual' in params:
                        query = query %(params, plant, plant, grouping)
                    elif 'forecast' in params:
                        query = query %(params, plant, plant, plant, plant, plant, grouping)
                    else:
                        query = query %(time_interval, params, repl_time, plant, plant, grouping)
                elif 'category' in key or 'reason' in key:
                    if 'actual' in params or 'forecast' in params:
                        query = query %(params, plant, params, plant)
                    elif 'monthly' in key:
                        query = query %(params, plant, params, plant)
                    else:
                        query = query %(time_interval, params, repl_time, plant, time_interval, params, repl_time, plant)
                else:
                    query = query %(params, plant, grouping, params, plant, grouping)
            elif 'recovery' in key:
                if 'category' in key:
                    query = query %(params, send_back_type, plant, params, send_back_type, plant)
                else:
                    if combo['view'] == 'product_overall':
                        query = query %(params, send_back_type, plant, send_back_type, plant, grouping)
                    else:
                        query = query %(params, send_back_type, plant, send_back_type, plant, send_back_type, plant, grouping)
                        if combo['view'] == 'product_breakdown_overall':
                            query = f"SELECT * FROM ({query})"
            elif 'reason' in key:
                query = query %(params, send_back_type, params, send_back_type)
            elif 'category' in key:
                query = query %(params, send_back_type, plant, params, send_back_type, plant)
            elif 'style' in key:
                query = query %(params, calculation, "('LA', 'CHI', 'N/A')", grouping)
            else:
                query = query %(params, calculation, plant, grouping)
            # Define master query for each tab.
            if item+1 == total:
                master_query = f"{master_query} UNION {query} ORDER BY 1 DESC;"
            else:
                if item == 0:
                    master_query = query
                else:
                    master_query = f"{master_query} UNION {query}"
#             print(combo)
#             print(query)
#             print()
#     print(master_query)
#     print()
    # Query DB with master query.
    tab_df = pd.read_sql(master_query, conn)
    
    # Detemine which workbook to use.
    if 'reason' in key:
        if 'replacement' in key:
            workbook = workbook4
        else:
            workbook = workbook2
    elif 'style' in key:
        if 'replacement' in key:
            workbook = workbook5
        else:
            workbook = workbook3
    else:
        if 'recovery' in key:
            workbook = workbook6
        else:
            workbook = workbook1
    
    # Final data munging in pandas based on whether weekly/monthly.
    if 'weekly' in key:
        # Add 21 days to week_ended for forecast only.
        if 'replacement' not in key or 'recovery' not in key:
            tab_df['week_ended'] = tab_df.apply(lambda row: date_checker(row['view'], row['week_ended'], 'week'), axis=1)
        
        ### Add category to uid, just one
        if 'category' in key:
            if 'repl_category_dashboard' in tab_df.columns:
                tab_df = tab_df.rename(columns={'repl_category_dashboard': 'master_category'})
            keys.insert(0, 'week_ended')
            keys.append('master_category')
            if 'recovery' in key:
                keys.insert(2, 'product')
                tab_df['master_category'] = tab_df['master_category'].replace(np.nan, 'N/A', regex=True)
            uid = tab_df[keys].astype(str).agg('_'.join, axis=1)
            tab_df.insert(loc=0, column='uid', value=uid)
        else:    
            # Concat first four columns.
            keys.insert(0, 'week_ended')
            if key == 'replacement_style_weekly':
                tab_df = tab_df.drop(columns=['plant'])
                keys.remove('plant')
            uid = tab_df[keys].astype(str).agg('_'.join, axis=1)
            tab_df.insert(loc=0, column='uid', value=uid)
            ### Add in second concat for master reason.
            if 'reason' in key:
                keys.append('master_reason')
                uid2 = tab_df[keys].astype(str).agg('_'.join, axis=1)
                tab_df.insert(loc=0, column='uid2', value=uid2)

        # Make sure that we are not returning rows with date greater than current EOW.
#         if key == 'send_back_product_size_weekly' or key == 'send_back_style_weekly':
        tab_df = tab_df[((tab_df['week_ended'] <= current_eow) & (tab_df['week_ended'] >= starting_55_eow)) & (pd.isnull(tab_df.rate) == False)]
        tab_df = tab_df.sort_values(by='week_ended', ascending=False)
    #         else:
#             tab_df = tab_df[tab_df['week_ended'] <= current_eow]
        
    elif 'monthly' in key:
        # Add 21 days to month_ended for forecast only.
    #         tab_df['month_ended'] = tab_df.apply(lambda row: date_checker(row['view'], row['month_ended'], 'month'), axis=1)
        if 'week_ended' in tab_df.columns:
            tab_df = tab_df.rename(columns={'week_ended': 'month_ended'})
            
        ### Add category to uid, just one
        if 'category' in key:
            if 'repl_category_dashboard' in tab_df.columns:
                tab_df = tab_df.rename(columns={'repl_category_dashboard': 'master_category'})
            keys.insert(0, 'month_ended')
            keys.append('master_category')
            if 'recovery' in key:
                keys.insert(2, 'product')
                tab_df['master_category'] = tab_df['master_category'].replace(np.nan, 'N/A', regex=True)
            uid = tab_df[keys].astype(str).agg('_'.join, axis=1)
            tab_df.insert(loc=0, column='uid', value=uid)
        else:    
            # Concat first four columns.
            keys.insert(0, 'month_ended')
            if key == 'replacement_style_monthly':
                keys.remove('plant')
            uid = tab_df[keys].astype(str).agg('_'.join, axis=1)
            tab_df.insert(loc=0, column='uid', value=uid)
            ### Add in second concat for master reason.
            if 'reason' in key:
                keys.append('master_reason')
                uid2 = tab_df[keys].astype(str).agg('_'.join, axis=1)
                tab_df.insert(loc=0, column='uid2', value=uid2)
        tab_df = tab_df[(tab_df['month_ended'] >= last_month) & (pd.isnull(tab_df.rate) == False)]
        tab_df = tab_df.sort_values(by='month_ended', ascending=False)
    # Clear all data from current worksheet.
    clear_data_range(workbook, key)
#     print(len(tab_df))
    try:
        # Update worksheet with new data, if error, print to terminal.
        pd_to_gsheets(workbook, key, tab_df, 1, 1)
        print(f"Completed update on tab {key}")
    except:
        ### Consider adding email generator to send raised exception message details.
        print(f"Failed to update {key}")
        print()
        continue
    print()

send_back_product_weekly
https://docs.google.com/spreadsheets/d/1Y6D0-TRXMT3gwN3HITouUmMJrhUPqItTykXcCFfU23I/edit#gid=1763802414 - send_back_product_weekly: Cleared
Rows available in WS: 9575
Rows in DF: 9145
Completed update on tab send_back_product_weekly

send_back_product_monthly
https://docs.google.com/spreadsheets/d/1Y6D0-TRXMT3gwN3HITouUmMJrhUPqItTykXcCFfU23I/edit#gid=1763802414 - send_back_product_monthly: Cleared
Rows available in WS: 1345
Rows in DF: 1293
Completed update on tab send_back_product_monthly

send_back_product_size_weekly
https://docs.google.com/spreadsheets/d/1Y6D0-TRXMT3gwN3HITouUmMJrhUPqItTykXcCFfU23I/edit#gid=1763802414 - send_back_product_size_weekly: Cleared
Rows available in WS: 61097
Rows in DF: 58909
Completed update on tab send_back_product_size_weekly

send_back_product_size_monthly
https://docs.google.com/spreadsheets/d/1Y6D0-TRXMT3gwN3HITouUmMJrhUPqItTykXcCFfU23I/edit#gid=1763802414 - send_back_product_size_monthly: Cleared
Rows available in WS: 8847

In [24]:
# # Query paramters master dictionary.

# # Update paramters as needed.
# query_params = {
#     'send_back_product_weekly': {'view': {'actual': [weekly_actual_query, actual_qty], 'forecast': [weekly_forecast_query, weekly_forecast_qty]},
#                               'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
#                               'send_back__type': {'send_back_both': "('return', 'exchange')",
#                                                   'return': "('return')", 'exchange': "('exchange')"},
#                                 'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}},
#     'send_back_product_monthly': {'view': {'actual': [monthly_actual_query, actual_qty], 'forecast': [monthly_forecast_query, monthly_forecast_qty]},
#                           'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
#                           'send_back__type': {'send_back_both': "('return', 'exchange')",
#                                               'return': "('return')", 'exchange': "('exchange')"},
#                             'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}},
#     'send_back_product_size_weekly': {'view': {'actual': [weekly_actual_query, actual_qty], 'forecast': [weekly_forecast_query, weekly_forecast_qty]},
#                               'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
#                               'send_back__type': {'send_back_both': "('return', 'exchange')",
#                                                   'return': "('return')", 'exchange': "('exchange')"},
#                                 'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"},
#                                 'size': {'all': "'all' size", 'size_categories': sizes}},
#     'send_back_product_size_monthly': {'view': {'actual': [monthly_actual_query, actual_qty], 'forecast': [monthly_forecast_query, monthly_forecast_qty]},
#                               'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
#                               'send_back__type': {'send_back_both': "('return', 'exchange')",
#                                                   'return': "('return')", 'exchange': "('exchange')"},
#                                 'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"},
#                                 'size': {'all': "'all' size", 'size_categories': sizes}},
#     'send_back_product_category_weekly': {'view': {'actual': weekly_actual_category_query, 'forecast': weekly_forecast_category_query},
#                               'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
#                               'send_back__type': {'send_back_both': "('return', 'exchange')",
#                                                   'return': "('return')", 'exchange': "('exchange')"},
#                                 'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}
#                                          },
#     'send_back_product_category_monthly': {'view': {'actual': monthly_actual_category_query, 'forecast': monthly_forecast_category_query},
#                               'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
#                               'send_back__type': {'send_back_both': "('return', 'exchange')",
#                                                   'return': "('return')", 'exchange': "('exchange')"},
#                                 'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}
#                                           },
#     'send_back_product_size_reason_weekly': {'view': {'actual': weekly_actual_reason_query, 'forecast': weekly_forecast_reason_query},
#                               'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
#                               'send_back__type': {'send_back_both': "('return', 'exchange')",
#                                                   'return': "('return')", 'exchange': "('exchange')"},
#                                 'size': {'all': "'all' size", 'size_categories': sizes}
#                                             },
#     'send_back_product_size_reason_monthly': {'view': {'actual': monthly_actual_reason_query, 'forecast': monthly_forecast_reason_query},
#                               'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
#                               'send_back__type': {'send_back_both': "('return', 'exchange')",
#                                                   'return': "('return')", 'exchange': "('exchange')"},
#                                 'size': {'all': "'all' size", 'size_categories': sizes}
#                                              },
#     'send_back_style_weekly': {'view': {'actual': [weekly_actual_query, actual_qty_style], 'forecast': [weekly_forecast_query, weekly_forecast_qty_style]},
#                               'rug_name': {'rug_name': " design_name rug_name"},
#                               'send_back__type': {'send_back_both': "('return', 'exchange')",
#                                                   'return': "('return')", 'exchange': "('exchange')"}},
#     'send_back_style_monthly': {'view': {'actual': [monthly_actual_query, actual_qty_style], 'forecast': [monthly_forecast_query, monthly_forecast_qty_style]},
#                               'rug_name': {'rug_name': " design_name rug_name"},
#                               'send_back__type': {'send_back_both': "('return', 'exchange')",
#                                                   'return': "('return')", 'exchange': "('exchange')"}},
# #     'replacement_product_weekly': {'view': {'30_Day': [weekly_replacement_product, 30], '90_Day': [weekly_replacement_product, 90], '240_Day': [weekly_replacement_product, 240]},
# #                               'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
# #                               'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}},
# #     'replacement_product_monthly': {'view': {'30_Day': [monthly_replacement_product, 30], '90_Day': [monthly_replacement_product, 90], '240_Day': [monthly_replacement_product, 240]},
# #                               'product': {'all': "'all' product", 'categories': product_categories, 'cushioned': product_cushioned},
# #                               'plant': {'all': "('LA', 'CHI', 'N/A')", 'LA': "('LA')", 'CHI': "('CHI')", 'N/A': "('N/A')"}}
# }

In [25]:
# '30_Day': [weekly_replacement_product_time, 30, "('_0_30')"], '90_Day': [weekly_replacement_product_time, 90, "('_0_30', '_31_90')"]